In [1]:
%autosave 0

Autosave disabled


In [2]:
!pip install grpcio tensorflow-serving-api==2.17.0

  Using cached tensorflow_serving_api-2.17.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached grpcio-1.68.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.3 MB)
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_

In [3]:
!pip install keras-image-helper

In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2024-12-16 20:43:48.013416: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 20:43:48.045873: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 20:43:48.114362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734381828.169546   23987 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734381828.183063   23987 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 20:43:48.253864: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [8]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [9]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [10]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['inputs'].CopyFrom(np_to_protobuf(X))

In [11]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.FAILED_PRECONDITION
	details = "Could not find variable conv2d_63/kernel. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status error message=Resource localhost/conv2d_63/kernel/N10tensorflow3VarE does not exist.
	 [[{{function_node __inference_serving_default_3416}}{{node functional_14_1/xception_1/conv2d_63_1/convolution/ReadVariableOp}}]]"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:8500 {created_time:"2024-12-16T20:44:02.376130246+00:00", grpc_status:9, grpc_message:"Could not find variable conv2d_63/kernel. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status error message=Resource localhost/conv2d_63/kernel/N10tensorflow3VarE does not exist.\n\t [[{{function_node __inference_servin

In [ ]:
preds = pb_response.outputs['output_0'].float_val

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [ ]:
dict(zip(classes, preds))